# Data Wrangling & Cleaning

In [ ]:
# import the library
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# convert scientific notation to decimals
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Load & Merge the data

In [ ]:
df_listing = pd.read_csv('data/kc_house_data.csv')
df_walking_score = pd.read_csv('data/walking_score.csv')
df_income = pd.read_csv('data/ZIP-3.csv')

### Summarizing your data for inspection

In [ ]:
print('Listings')
print(df_listing.columns)
print(df_listing.head())
print(df_listing.describe())
print('')
print('Walking Score')
# TODO: print the columns, head and describe for the Walking Score dataframe
print('')
print('Income')
# TODO: print the columns, head and describe for the Income dataframe

### Fixing column name

In [ ]:
df_income.columns = ['zipcode', 'median_income', 'mean_income', 'population']

### Converting data types

In [ ]:
df_listing['date'] = pd.to_datetime(df_listing['date'])
df_income['median_income'] = df_income['median_income'].str.replace(',', '').astype(float)
df_income['mean_income'] = df_income['mean_income'].str.replace(',', '').astype(float)
df_income.head()

In [ ]:
# TODO: Convert the data type of the population column
df_income

### Dealing with missing values
How to deal with the missing values? Should we remove the rows or fill the gap with a value?

In [ ]:
# Number of missing values by columns
print(df_listing.isnull().sum())
print('')
print(df_walking_score.isnull().sum())
print('')
print(df_income.isnull().sum())

In [ ]:
# select all the rows with missing values
df_walking_score[df_walking_score.isnull().any(axis=1)]

In [ ]:
# select all the rows with missing values
df_income[df_income.isnull().any(axis=1)]

In [ ]:
# TODO: Create a strategy to handle the missing values on the Walking Score and Income dataframes

### Removing outliers
Some algorithms are very sensitive to outliers. Considering the number of bedrooms, should we remove houses with an extreme number of bedrooms? How many bedrooms are too many? (Suggestion: as a rule of thumb, three standard deviations from the mean is a good measure to identify outliers).

In [ ]:
# bedrooms
print(df_listing['bedrooms'].value_counts())
print('mean', np.mean(df_listing['bedrooms']))
print('std', np.std(df_listing['bedrooms']))
plt.hist(df_listing['bedrooms'], bins=20)
plt.show()

In [ ]:
# TODO: Remove the outlier houses considering the number of bedrooms

In [ ]:
# Dealing with outliers
houses_to_remove = []

# remove based on zipcode and price
for zipcode in df_listing['zipcode'].unique():
    df_zipcode = df_listing[df_listing['zipcode']==zipcode]
    m = np.mean(df_zipcode['price'])
    s = np.std(df_zipcode['price'])
    houses_to_remove = houses_to_remove + list(df_zipcode[df_zipcode['price']>m+3.0*s].index)
print('')
print('# houses to remove', len(houses_to_remove))

df_listing = df_listing[~df_listing.index.isin(houses_to_remove)]

### Merging Data Sets

In [ ]:
df_merge = df_listing.copy()
df_merge = df_merge.merge(df_walking_score, on='zipcode', how='left')
df_merge = df_merge.merge(df_income, on='zipcode', how='left')

In [ ]:
print('Total # houses', len(df_merge))

### Saving the processed file

In [ ]:
df_merge.to_csv('data/house_pricing.csv', index=False)